In [1]:
from ipywidgets import FloatSlider, interact, SelectMultiple, IntSlider
from IPython.display import display, clear_output
from model_scoring_func import get_all_names
from optimisation import min_max_df, opt_with_starts, my_g, formulation_result, property_results
from datahandling import access_db
from pandas import DataFrame
from numpy import append
from itertools import izip_longest

In [2]:
ingredients = ['PVC', 'filler', 'FR', 'stabiliser', 'DINP', 'LDH', 'spherical_filler']
ingr = [i + ' Cost' for i in ingredients]
ingr_cost_ini = [15.0, 8.0, 48.0, 45.0, 30.0, 40.0, 3.0]
ingr_cost_ini_dict = dict(zip(ingr, ingr_cost_ini))

In [3]:
options = list(get_all_names())

In [4]:
mins = min_max_df(options)['min'].values.round()
maxs = min_max_df(options)['max'].values.round()

In [5]:
options += ingr

In [6]:
mod_db = access_db(3, True)
mod_df = DataFrame(mod_db.all())
low_high_list = mod_df.desired_vals.values

Initiate widget for selecting which inputs to Change

In [7]:
selection = SelectMultiple(options=options)

Initiate slider widgets for properties and costs of ingredients

In [8]:
w_list = []
for name, my_min, my_max in izip_longest(options, mins, maxs):
    if my_min is None:
        cost = ingr_cost_ini_dict[name]
        w_list.append(IntSlider(value=cost, min=0, max=60, description=name, visible=False))
        continue
    
    w_list.append(FloatSlider(min=my_min, max=my_max, step=1.0, description=name, visible=False))

Link selection ot slider widgets

In [9]:
def selection_change(*args):
    for w, lh in izip_longest(w_list, low_high_list):
        if w.description in selection.value:
            w.visible = True
        elif lh == 'high':
            w.visible = False
            w.value = w.min
        elif lh == 'low':
            w.visible = False
            w.value = w.max
        else:
            w.visible = False
            w.value = ingr_cost_ini_dict[w.description]

In [10]:
selection.observe(selection_change, 'value')

Define optimisation function

In [11]:
def optimisation(**kwargs):
#     clear_output()
    print('Calculating...')
    
    con_list = []
    con_limits = []
    ingr_cost = []
    for name, my_min, my_max in izip_longest(options, mins, maxs):
        val = kwargs[name]
        if val != my_min and val != my_max and my_min != None:
            con_list.append(name)
            con_limits.append(val)
        elif my_min == None:
            ingr_cost.append(val)
            
#     print(con_list)
#     print(con_limits)
#     print(ingr_cost)

    cons = ({'type': 'eq',
             'fun': lambda x: sum(x) - 1},
            {'type': 'ineq', 
             'fun': my_g, 
             'args': [con_list, con_limits]})
    
    bnds = [[0.0, 1.0] for i in range(7)]
    bnds[0] = [0.4, 0.8]
    
    best_res = opt_with_starts(ingr_cost, bnds, cons, starts=3)
    
    clear_output()
    print(best_res.message)
    print('Optimum Cost of Formulation: {} R/kg'.format(round(best_res.fun, 2)))
    
    form_df = formulation_result(best_res.x)
    display(form_df)
    
    prop_df = property_results(best_res.x, con_list, con_limits)
    display(prop_df)

In [12]:
w_dict = dict(((w.description, w) for w in w_list))

#### Interface

Select which properties to constrain and which ingredients costs to change

Ingredient costs which are not selected are set at the initial values

In [13]:
display(selection)
interact(optimisation, __manual=True, **w_dict)

Optimization terminated successfully.
Optimum Cost of Formulation: 18.76 R/kg


,mass_frac_%
PVC,48.78
filler,13.08
FR,6.69
stabiliser,0.98
DINP,18.32
LDH,2.40
spherical_filler,9.76
